Dan Shea  
2017.05.04  

lncRNA analysis of RNA-seq data for _B. rapa_ subsp. _pekinensis_ var. S11  
We have a `tmap` file generated by `stringtie`, `gffcompare`, and the differential expression analysis from `cuffcompare`. Additionally, we have the `gene_exp.diff` file that tells us the loci of the transfrags. We want to divide the transfrags into different types based on their mapping to gene loci of the _B. rapa_ var. Chiifu-401 reference sequence and additionally create fasta files of the genomic DNA sequences of the transcripts for use in single-strand `blastx` analysis against the Swissprot database. This should allow us to confirm both putative novel genes and putative lncRNAs such as lincRNA (long-intergenic), cis-NAT (opposite strand of gene locus), and intronic ncRNA (within the intron of a gene locus.

In [1]:
# Let's import some of the packages we will require for this analysis
import pandas as pd
from Bio import SeqIO

In [2]:
# Define the input files we want to load in as pandas data frames
gene_exp_file = 'gene_exp.diff'
tmap_file     = 'merged.stringtie_merged.gtf.tmap'

In [3]:
# Load the files as pandas data frames
gene_exp = pd.read_table(gene_exp_file)
tmap     = pd.read_table(tmap_file)

In [4]:
gene_exp

,test_id,gene_id,gene,locus,sample_1,sample_2,status,value_1,value_2,log2(fold_change),test_stat,p_value,q_value,significant
0,Bra000001,Bra000001,Bra000001,A03:8794510-8797095,s11_NV,s11_4V,NOTEST,0.026202,0.000000,-inf,0.000000,1.00000,1.000000,no
1,Bra000002,Bra000002,Bra000002,A03:8797898-8800379,s11_NV,s11_4V,NOTEST,0.000000,0.000000,0.000000,0.000000,1.00000,1.000000,no
2,Bra000007,Bra000007,Bra000007,A03:8823496-8824207,s11_NV,s11_4V,OK,0.526043,1.645720,1.645470,1.755550,0.02835,0.074204,no
3,Bra000008,Bra000008,Bra000008,A03:8829307-8830496,s11_NV,s11_4V,NOTEST,0.114528,0.090949,-0.332571,0.000000,1.00000,1.000000,no
4,Bra000010,Bra000010,Bra000010,A03:8840534-8842281,s11_NV,s11_4V,NOTEST,0.157205,0.206450,0.393143,0.000000,1.00000,1.000000,no
5,Bra000016,Bra000016,Bra000016,A03:8884292-8886197,s11_NV,s11_4V,NOTEST,0.000000,0.000000,0.000000,0.000000,1.00000,1.000000,no
6,Bra000020,Bra000020,Bra000020,A03:8897927-8898629,s11_NV,s11_4V,NOTEST,0.000000,0.000000,0.000000,0.000000,1.00000,1.000000,no
7,Bra000022,Bra000022,Bra000022,A03:8903597-8904350,s11_NV,s11_4V,NOTEST,0.225523,0.033750,-2.740320,0.000000,1.00000,1.000000,no
8,Bra000033,Bra000033,Bra000033,A03:8941847-8945170,s11_NV,s11_4V,OK,0.711796,0.040692,-4.128640,-2.721520,0.07485,0.159078,no
9,Bra000038,Bra000038,Bra000038,A03:8982374-8984291,s11_NV,s11_4V,OK,0.670964,0.651794,-0.041819,-0.046157,0.95035,0.968829,no


In `gene_exp`, entries where the `gene` column have been mapped to `-` means that the transcript did not map to a gene, but it might still be mapped at a gene locus. It may either have mapped to the opposing strand at a gene locus, or it mapped to an intron.

In [5]:
tmap

,ref_gene_id,ref_id,class_code,qry_gene_id,qry_id,FMI,FPKM,FPKM_conf_lo,FPKM_conf_hi,cov,len,major_iso_id,ref_match_len
0,Bra011901,Bra011901,=,Bra011901,Bra011901,100,0.0,0.0,0.0,0.0,774,Bra011901,774
1,-,-,u,MSTRG.2,MSTRG.2.1,100,0.0,0.0,0.0,0.0,730,MSTRG.2.1,-
2,Bra011898,Bra011898,=,Bra011898,Bra011898,100,0.0,0.0,0.0,0.0,969,Bra011898,969
3,Bra011899,Bra011899,=,Bra011899,Bra011899,100,0.0,0.0,0.0,0.0,810,Bra011899,810
4,Bra011891,Bra011891,=,Bra011891,Bra011891,100,0.0,0.0,0.0,0.0,297,Bra011891,297
5,Bra011902,Bra011902,x,MSTRG.6,MSTRG.6.1,100,0.0,0.0,0.0,0.0,929,MSTRG.6.1,1299
6,Bra011902,Bra011902,=,Bra011902,Bra011902,100,0.0,0.0,0.0,0.0,1299,Bra011902,1299
7,Bra011889,Bra011889,=,Bra011889,Bra011889,100,0.0,0.0,0.0,0.0,318,Bra011889,318
8,Bra011897,Bra011897,=,Bra011897,Bra011897,100,0.0,0.0,0.0,0.0,1695,Bra011897,1695
9,Bra011896,Bra011896,p,MSTRG.10,MSTRG.10.1,100,0.0,0.0,0.0,0.0,393,MSTRG.10.1,981


`class_code` defined in the tmap can have one of the following values: 

<table>
<tr><th>Priority</th><th>Code</th><th>Description</th></tr>
<tr><td>1</td><td>=</td><td>Complete match of intron chain</td></tr>
<tr><td>2</td><td>c</td><td>Contained</td></tr>
<tr><td>3</td><td>j</td><td>Potentially novel isoform (fragment): at least one splice junction is shared with a reference transcript</td></tr>
<tr><td>4</td><td>e</td><td>Single exon transfrag overlapping a reference exon and at least 10 bp of a reference intron, indicating a possible pre-mRNA fragment.</td></tr>
<tr><td>5</td><td>i</td><td>A transfrag falling entirely within a reference intron</td></tr>
<tr><td>6</td><td>o</td><td>Generic exonic overlap with a reference transcript</td></tr>
<tr><td>7</td><td>p</td><td>Possible polymerase run-on fragment (within 2Kbases of a reference transcript)</td></tr>
<tr><td>8</td><td>r</td><td>Repeat. Currently determined by looking at the soft-masked reference sequence and applied to transcripts where at least 50% of the bases are lower case</td></tr>
<tr><td>9</td><td>u</td><td>Unknown, intergenic transcript</td></tr>
<tr><td>10</td><td>x</td><td>Exonic overlap with reference on the opposite strand</td></tr>
<tr><td>11</td><td>s</td><td>An intron of the transfrag overlaps a reference intron on the opposite strand (likely due to read mapping errors)</td></tr>
</table>

In [6]:
# First we want to acquire all entries in gene_exp that mapped to '-'
putative_lncrna = gene_exp[gene_exp.gene == '-']

In [7]:
# Then, we want to find out the class_code assigned to those transcripts.
putative_lncrna_complete = pd.merge(putative_lncrna, tmap, left_on='test_id', right_on='qry_gene_id')

In [8]:
# the merge creates a row for each isoform, we only want the major_iso_id
putative_lncrna_complete = putative_lncrna_complete[putative_lncrna_complete.major_iso_id == putative_lncrna_complete.qry_id]

In [9]:
# Now subset them based on the value of the class_code
class_codes = set(putative_lncrna_complete.class_code)

In [10]:
class_codes

{'e', 'i', 'o', 'p', 's', 'u', 'x'}

In [11]:
# Here, we store the entries into a dictionary of lists, where the key is the assigned class code
putative_lncrna_complete_dict = {class_code: None for class_code in class_codes}
for class_code in class_codes:
    putative_lncrna_complete_dict[class_code] = putative_lncrna_complete[putative_lncrna_complete.class_code == class_code]    

In [12]:
# Acquire the counts for each class_code
for class_code in class_codes:
    print('Number of class {}: {}'.format(class_code, putative_lncrna_complete_dict[class_code].shape[0]))

Number of class s: 12
Number of class p: 274
Number of class u: 2124
Number of class x: 560
Number of class i: 109
Number of class e: 1
Number of class o: 10


Now we can filter based on likely derivations of the transcripts.

* We __don't want__ pre-mRNA fragments (class_code 'e')
* We __don't want__ generic exonic overlaps (class_code 'o')
* We __don't want__ likely read mapping errors (class_code 's')
* We __don't want__ possible polymerase run-ons (class_code 'p')

However, we __do want__ the following:

* A transfrag falling entirely within a reference intron (class_code 'i')
* Exonic overlap with reference on the opposite strand (class_code 'x')
* Unknown, intergenic transcript (class_code 'u')

In [13]:
# Let's store the results into variables, so that we may create fasta files for the sequences and print out
# tab-delimited files for creating Excel spreadsheet tables later
intronic   = putative_lncrna_complete_dict['i']
NAT        = putative_lncrna_complete_dict['x']
intergenic = putative_lncrna_complete_dict['u']

In [14]:
# Let's load in the B. rapa reference file and create fasta files of the gDNA sequences
reference_file = '../reference/fasta/Brapa_v1.5.fa'
reference_io   = SeqIO.parse(reference_file, 'fasta')
# Load the reference into a big dictionary for later use and re-use
reference = dict()
for seq in reference_io:
    reference[seq.id] = seq

In [15]:
# Now, for each class we do want, we extract the relevant sequences and then output them to fasta files
# We want to extract the query_id and the locus and parse the locus into chromosome and start/stop coordinates
intronic_records = list()
for index, (test_id, locus) in intronic[['test_id', 'locus']].iterrows():
    # Split the locus into chromosome and coordinates
    (chromosome, coords) = locus.split(':')
    # Split the coordinates into start and stop coordinates
    (start, stop)        = coords.split('-')
    # convert the start and stop coordinates to integer values
    # Note gene_exp.diff seems to already provide python based coordinates
    start = int(start)
    stop  = int(stop)
    # Now, for each entry get the gDNA sequence and save it as a SeqRecord
    # The list of SeqRecords will be written out to a fasta file
    # We also update the id, name and description of the record
    temp_seqRecord = reference[chromosome][start:stop]
    temp_seqRecord.id = test_id
    temp_seqRecord.name = test_id
    temp_seqRecord.description = '{}:{}-{}'.format(chromosome, start+1, stop-1)
    intronic_records.append(temp_seqRecord)
    
SeqIO.write(intronic_records, 'putative_intronic_lncRNAs.fasta', format='fasta')

109

In [16]:
NAT_records = list()
for index, (test_id, locus) in NAT[['test_id', 'locus']].iterrows():
    # Split the locus into chromosome and coordinates
    (chromosome, coords) = locus.split(':')
    # Split the coordinates into start and stop coordinates
    (start, stop)        = coords.split('-')
    # convert the start and stop coordinates to integer values
    # Note gene_exp.diff seems to already provide python based coordinates
    start = int(start)
    stop  = int(stop)
    # Now, for each entry get the gDNA sequence and save it as a SeqRecord
    # The list of SeqRecords will be written out to a fasta file
    # We also update the id, name and description of the record
    temp_seqRecord = reference[chromosome][start:stop]
    temp_seqRecord.id = test_id
    temp_seqRecord.name = test_id
    temp_seqRecord.description = '{}:{}-{}'.format(chromosome, start+1, stop-1)
    NAT_records.append(temp_seqRecord)
    
SeqIO.write(NAT_records, 'putative_NAT_lncRNAs.fasta', format='fasta')

560

In [17]:
intergenic_records = list()
for index, (test_id, locus) in intergenic[['test_id', 'locus']].iterrows():
    # Split the locus into chromosome and coordinates
    (chromosome, coords) = locus.split(':')
    # Split the coordinates into start and stop coordinates
    (start, stop)        = coords.split('-')
    # convert the start and stop coordinates to integer values
    # Note gene_exp.diff seems to already provide python based coordinates
    start = int(start)
    stop  = int(stop)
    # Now, for each entry get the gDNA sequence and save it as a SeqRecord
    # The list of SeqRecords will be written out to a fasta file
    # We also update the id, name and description of the record
    temp_seqRecord = reference[chromosome][start:stop]
    temp_seqRecord.id = test_id
    temp_seqRecord.name = test_id
    temp_seqRecord.description = '{}:{}-{}'.format(chromosome, start+1, stop-1)
    intergenic_records.append(temp_seqRecord)
    
SeqIO.write(intergenic_records, 'putative_intergenic_lncRNAs.fasta', format='fasta')

2124

In [18]:
# Finally, we now output the table data for each class_code we created a fasta file for as an excel file
writer = pd.ExcelWriter('putative_lncrnas.xlsx')
intronic.to_excel(writer, 'Intronic')
NAT.to_excel(writer, 'NAT')
intergenic.to_excel(writer, 'Intergenic')
writer.save()